In [9]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from adversarial_dataset import AdversarialDataset
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchvision import models
from utils import correct_torch_image, minmax_scaler, denormalize
import numpy as np

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

noise_transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = AdversarialDataset(annotation_file="images.csv",
                             categories_file="categories.csv",
                             img_dir="images",
                             noise_dir="adversarial_noise_targeted",
                             img_transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model = model.to(device)
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [14]:
n_correct, n_adv_correct, n_adv_targets_correct = 0, 0, 0

for image, noise, label, target, id in tqdm(dataloader):
    image = image.to(device)
    noise = noise.to(device)
    label = label.to(device)
    target = target.to(device)
    
    org_output = model(image)
    _, org_predictions = torch.max(org_output, dim=1)
    n_correct += torch.sum(org_predictions == label)
    
    adv_output = model(image + noise)
    _, adv_predictions = torch.max(adv_output, dim=1)
    n_adv_correct += torch.sum(adv_predictions == label)
    n_adv_targets_correct += torch.sum(adv_predictions == target)
    

n_correct, n_adv_correct, n_adv_targets_correct

  0%|          | 0/32 [00:00<?, ?it/s]

(tensor(951, device='cuda:0'),
 tensor(51, device='cuda:0'),
 tensor(946, device='cuda:0'))